In [1]:
import pandas as pd
import numpy as np
import os
import json
# import dt
import re

# GM_RESULT

In [2]:
res = pd.read_csv("game_results.csv")
res.head()


,game_id,seed_game_id,players,generations,game_options,scores,createtime
0,f7cbeb6cc687,NaN,4,8,NaN,"[{""corporation"":""Poseidon"",""playerScore"":69,""p...",31/8/2020 15:05:21
1,10d3f58364b6,NaN,2,12,NaN,"[{""corporation"":""Aridor"",""playerScore"":144,""pl...",31/8/2020 15:05:21
2,ff2c9f3f3e45,NaN,4,8,NaN,"[{""corporation"":""Inventrix (breakthrough)"",""pl...",31/8/2020 15:05:21
3,cf45c3f6faa2,NaN,2,11,NaN,"[{""corporation"":""Arklight (breakthrough)"",""pla...",31/8/2020 15:05:21
4,79cef4b6a297,NaN,2,8,NaN,"[{""corporation"":""Point Luna"",""playerScore"":60,...",31/8/2020 15:05:21


In [3]:
# preprocessing
res["scores"] = res["scores"].astype(object)
res["game_options"] = res["game_options"].astype(object)
res["createtime"] = pd.to_datetime(res["createtime"], dayfirst=True)


In [4]:
# scores proprocessing
store = []
for i in res["scores"]:
    tmp = json.loads(i)
    for i in range(len(tmp)):
        tmp[i]['order'] = i+1
    tmp = sorted(tmp,key = lambda e:e.__getitem__('playerScore'),reverse = True) # can optimize to obtain knowledge of order
    for i in range(len(tmp)):
        tmp[i]['position'] = i+1
    store.append(tmp)
df1 = pd.DataFrame(store)
df1.columns = ["player1","player2","player3","player4","player5","player6"]

In [5]:
options = []
for i in res["game_options"]:
    if not pd.isna(i):
        options.append(json.loads(i))
    else: options.append({})
df2 = pd.DataFrame(options)

In [6]:
# df2.head()

In [7]:
# df3 = pd.concat([res, df1, df2],axis=1)
# df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers','politicalAgendasExtensionToggle','aresHazards','requiresVenusTrackCompletion','politicalAgendasExtensionToggle','solarPhaseOption'])

In [8]:
df3 = pd.concat([res, df1, df2],axis=1)
df4= df3.drop(columns=['seed_game_id','game_options','scores','draftVariant','corporateEra','solarPhaseOption','undoOption','fastModeOption','includeVenusMA','soloTR','removeNegativeGlobalEventsOption','politicalAgendasExtension','requiresMoonTrackCompletion','showTimers'])

In [9]:
df4.columns
df4.to_csv('./prprocessing_data', index = False)

In [10]:
def getPlayer(df,num): # get all players name
    player_df = pd.DataFrame(columns=['player','count'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                player_df.loc[player_df.shape[0]] = [i[column]['player'].lower().strip(),1]
    ff1 = player_df.groupby(player_df["player"]).count()
    ff1["player_list"] = ff1.values.tolist()
    return ff1


In [11]:
player_count = getPlayer(df4,4)
player_count.to_csv('./pl_count.csv', index = True)

In [12]:
def playerCount(df,num,player): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        if i['players'] == num: #start
            for column in ['player1','player2','player3','player4']:
                if i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' not in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                elif i[column]['player'].lower() in [p.lower() for p in player] and ('corporation2' in i[column].keys()):
                    player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
    return player_df

In [13]:
df_player = playerCount(df4,4,['ee'])
df_player.tail(20)

,id,corporation1,rank,order,score,time,generations,corporation2
0,ee,Aphrodite (breakthrough),2,1,82,2021-04-26 16:26:26,7,nan
1,ee,Manutech,1,1,74,2021-04-27 17:34:43,7,nan
2,ee,Inventrix (breakthrough),4,2,76,2021-04-28 01:41:52,8,nan
3,ee,Terralabs Research (breakthrough),1,1,85,2021-04-28 17:40:15,8,nan
4,ee,Philares,1,1,65,2021-04-29 01:01:53,5,nan
5,ee,Teractor (breakthrough),1,3,76,2021-04-30 01:36:51,7,nan
6,ee,Pharmacy Union,1,3,102,2021-05-20 00:42:47,7,nan
7,ee,Lakefront Resorts,2,2,66,2021-06-20 00:12:36,6,Mining guild (breakthrough)
8,ee,Philares,1,4,80,2021-07-03 14:13:03,6,Ecoline (breakthrough)
9,ee,Point Luna,1,3,87,2021-07-09 20:50:31,6,Teractor (breakthrough)


In [14]:
#time series
def time_series(df):
    ff1 = df[(df["players"]==4) & (df["generations"]<=9)]
    ff1.to_csv('./time_original', index = True)
    ff2 = ff1.groupby(ff1['createtime'].dt.date).mean()
    return ff2


In [15]:
# # 时间绘图
# df5 = time_series(df4)
# df5.head(20)
# df5 = df5.drop(columns=["players","initialDraftRounds"])
# df5.to_csv('./time_series.csv', index = True)
# # df5.plot()

# US_RESULT

In [16]:
# usage: demostrate user's creating time, like "you are the NO.1132 player"
user = pd.read_csv("users.csv")
user = user.drop(columns="prop")
user["name"] = user["name"].str.lower()
user["createtime"] = pd.to_datetime(user["createtime"], dayfirst=True)
user = user.sort_values(by=["createtime"])
user = user.reset_index(drop=True)
user["createtime"] = user.index+1
user.to_csv('./user_pre.csv', index = False)

In [17]:
# left join and mapping
user_player = user.merge(player_count,how="left",left_on="name",right_on="player")
user_player = user_player.sort_values(by=["player_list"],na_position="first")
user_player.to_csv('./user_player_mapping.csv', index = False)
user_player = user_player.set_index("name")
user_player = user_player.drop(columns="player_list")
user_player.tail()

,createtime,count
name,,
奶牛,84,165.0
阿尔,295,180.0
无语,22,189.0
钝钝,60,303.0
乙木,17,322.0


In [18]:
# detect missing user name
player_user = player_count.merge(user,how="left",left_on="player",right_on="name")
player_user = player_user.sort_values(by=["count"],ascending=False, na_position="first")
player_user = player_user[player_user["name"].isna()]
player_user.to_csv('./player_user_mapping.csv', index = False)
player_user.head()

,count,player_list,name,createtime
1548,254,[254],NaN,NaN
1398,220,[220],NaN,NaN
1549,124,[124],NaN,NaN
240,120,[120],NaN,NaN
947,114,[114],NaN,NaN


# CUSTOMIZATION

In [19]:
player_mapping_df = pd.read_csv('player_mapping.csv')
player_mapping_df = player_mapping_df.set_index("playername")

In [20]:
def playerGroup(df,num,player,map_df): #TODO,模糊查找
    player_df = pd.DataFrame(columns=['id','corporation1', 'rank', 'order','score', 'time', 'generations','corporation2'])
    for index,i in df.iterrows():
        try:
            if i['players'] == num: #start
                for column in ['player1','player2','player3','player4']:
                    if (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' not in i[column].keys()):
                        if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                            player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],'nan']
                    elif (i[column]['player'].lower() == player.lower() or i[column]['player'].lower() in map_df.index) and ('corporation2' in i[column].keys()):
                        if map_df.loc[i[column]['player'].lower(),'username']== player.lower():
                            player_df.loc[player_df.shape[0]] = [i[column]['player'].lower(),i[column]['corporation'],i[column]['position'],i[column]['order'],i[column]['playerScore'],i['createtime'],i['generations'],i[column]['corporation2']]
        except: print(player)
    return player_df

In [21]:
user_df = playerGroup(df4,4,'钝钝',player_mapping_df)
user_df.tail()

,id,corporation1,rank,order,score,time,generations,corporation2
336,钝钝,Thorgate (breakthrough),3,2,69,2021-08-23 22:45:31,7,Midas
337,钝钝,Inventrix (breakthrough),2,2,99,2021-08-25 02:34:41,7,Chaos
338,钝钝,Terralabs Research (breakthrough),1,2,155,2021-08-25 16:59:00,8,Incite Ender
339,钝钝,Manutech,1,4,100,2021-08-26 23:27:18,7,Utopia Invest
340,钝钝,Aridor,1,1,71,2021-08-29 23:09:41,5,Manutech


In [22]:
group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)

user_df["hour"] = user_df.time.apply(lambda x: x.hour)
group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
group_by_hour.reset_index().loc[0,'hour']
highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
# highest.head()


In [23]:
user_df.to_csv('钝钝.csv',index=False)

In [24]:
# test
firstgame = user_df.loc[0,["corporation1","rank","time"]]
user_df["date"] = user_df["time"].dt.normalize()
group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
group_by_date = group_by_date[group_by_date["id"]<=7]
# group_by_date.reset_index().head()

In [33]:
def user_summary(ff1,ff2,user): ## ff1=df4
    try:
        user_df = playerGroup(ff1,4,user,player_mapping_df)
    except:
        print(user)
        return 0
    # measure 0: check whether the account is uesable
    if user_df.empty: return [''for i in range(18)]
    if pd.isna(ff2.loc[user,"createtime"].astype("int")): return [''for i in range(18)]
    # measure 1: account create date
    createtime = ff2.loc[user,"createtime"]
    # measure 1.5: first game
    firstgame = user_df.loc[0,["corporation1","rank","time"]]
    # measure 1.6 spend how many days, defeat ?% users
    # measure 2: total games, favourite corp
    group_by_corp = user_df.groupby(['corporation1']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    group_by_corp = group_by_corp.reset_index()
    favorite_corp = group_by_corp.loc[0,'corporation1']
    favourite_corp_rank = group_by_corp.loc[0,'rank']
    favorite_corp_times = group_by_corp.loc[0,'id']
    games = user_df.shape[0]
    # measure 3: most frequently play time period
    user_df["hour"] = user_df.time.apply(lambda x: x.hour)
    group_by_hour = user_df.groupby(['hour']).agg({'id':'count','rank':'mean','score':'mean','generations':'mean'}).sort_values(by=['id'], ascending=False)
    most_freq_time = group_by_hour.reset_index().loc[0,'hour']
    # measure 4: avg score and gen, i.e. defeat 96% players (display this if the ratio > 50) TODO
    avg_rank = user_df["rank"].mean()
    avg_gens = user_df["generations"].mean()
    # measure 5: highest score, with which corp
    # highest = pd.to_numeric(user_df["score"]).idxmax()
    highest = user_df.sort_values(by=["score","generations"],ascending=[False,True])
    highest_corp = highest.iloc[0,1]
    highest_score = highest.iloc[0,4]
    highest_gens = highest.iloc[0,6]
    highest_corp = highest_corp+' 和 '+highest.iloc[0,7] if (highest.iloc[0,7] != 'nan' or highest.iloc[0,7] != '') else highest_corp
    # measure 6: group by day. i.e 2021-06-10 seems to be a special day for u, u played 6 games by a row
    user_df["date"] = user_df["time"].dt.normalize()
    group_by_date = user_df.groupby(['date']).agg({'id':'count'}).sort_values(by=['id'], ascending=False)
    group_by_date = group_by_date[group_by_date["id"]<=7]
    played_day = group_by_date.shape[0]
    group_by_date = group_by_date.reset_index()
    most_day = group_by_date.loc[0,["date","id"]]
    # print([user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]])
    return [user, firstgame[0],firstgame[1],firstgame[2],createtime, games,favorite_corp,favorite_corp_times,favourite_corp_rank,most_freq_time,avg_rank,avg_gens, highest_corp, highest_score,highest_gens,played_day,most_day[0], most_day[1]]

In [34]:
def measure_summary(ff1,ff2):
    output_df = pd.DataFrame(columns=["user_id","first_corp","first_rank","first_time","create_no","total_games","favorite_corp","fa_corp_times","fa_corp_rank","most_freq_play_time","avg_rank","avg_gens","highest_corp","highest_score","highest_gens","played_day","most_date","most_date_times"])
    for index,i in user_player.iterrows():
        # print(index)
        if (not pd.isnull(i["count"])) and i["count"]>=1 and (("tour" in index) == False):
            output_df.loc[output_df.shape[0]] = user_summary(ff1,ff2,index)
    return output_df

In [27]:
# def measure_summary(ff1,ff2):
#     output_df = pd.DataFrame(columns=["user_id"])
#     for index,i in user_player.iterrows():
#         output_df.loc[output_df.shape[0]] = [i]
#     return output_df
# test = measure_summary(df4,user_player)
# test = test.dropna()
# test.to_csv("test.csv")

In [35]:
print(user_summary(df4,user_player,'钝钝'))

['钝钝', 'Saturn Systems', 4, Timestamp('2020-08-31 15:05:21'), 60.0, 341, 'Point Luna', 24, 2.25, 23, 2.0146627565982405, 7.513196480938417, 'Trade Navigator and Midas', 208, 8, 201, Timestamp('2021-04-25 00:00:00'), 6]


In [29]:
output_df = measure_summary(df4,user_player)
output_df = output_df.dropna()
output_df.to_csv("measure_summary.csv")
output_df.head()


aydenegg
ccy


In [ ]:
def bayesian_avarage(rank,times):
    return (rank*times+2.5*10)/times+10

In [ ]:
output_df = pd.read_csv("measure_summary.csv")
# preprocessing
output_df = output_df[output_df["user_id"].str.contains("tour-")==False]
# play times
output_df = output_df.sort_values(by="total_games",ascending=False)
output_df['play_times_sort'] = range(1,len(output_df)+1)
# gens
output_df = output_df.sort_values(by="avg_gens",ascending=False)
output_df['gens_sort'] = range(1,len(output_df)+1)
# terraform
output_df = output_df.sort_values(by="avg_gens")
output_df['terra_sort'] = range(1,len(output_df)+1)
# terraform
output_df = output_df.sort_values(by="highest_score",ascending=False)
output_df['highest_sort'] = range(1,len(output_df)+1)
# rank
output_df["bayes_rank"] = output_df.apply(lambda x:(x["total_games"]*x["avg_rank"]+10*2.8)/(x["total_games"]+10),axis=1)
output_df = output_df.sort_values(by="bayes_rank")
output_df['rank_sort'] = range(1,len(output_df)+1)
# analyze corp ranking TODO: group by corp,ignore 'breakthrough', then find the index with the maximum values
output_df['baye_avg'] = output_df.apply(lambda x:(x["fa_corp_times"]*x["fa_corp_rank"]+5*2.5)/(x["fa_corp_times"]+5),axis=1)
output_df['max_corp'] = output_df.groupby(["favorite_corp"],as_index=False)["baye_avg"].transform(min)
output_df['max_corp'] = np.where((output_df["max_corp"]==output_df["baye_avg"]) & (output_df["baye_avg"] <= 2.5),1,0)
# output_pivot = pd.pivot_table(output_df, index=[''])
# output_df.sort_values(by="baye_avg").groupby(["favorite_corp"],as_index=False).transform(lambda x: x in range(1,len(output_df)+1))


output_df = output_df.sort_values(by="bayes_rank")
output_df.to_csv("measure_summary_sort.csv")
output_df.head()

,Unnamed: 0,user_id,first_corp,first_rank,first_time,create_no,total_games,favorite_corp,fa_corp_times,fa_corp_rank,...,most_date,most_date_times,play_times_sort,gens_sort,terra_sort,highest_sort,bayes_rank,rank_sort,baye_avg,max_corp
291,291,ender,Poseidon,4.0,2020-08-31 15:05:21,2.0,166.0,Manutech,12.0,1.583333,...,2021-08-25 00:00:00,2.0,11,261,35,22,1.812500,1,1.852941,1
265,265,文生,Lakefront Resorts,1.0,2020-10-06 17:10:48,7.0,41.0,Valley Trust (breakthrough),4.0,1.750000,...,2020-12-05 00:00:00,6.0,39,176,120,10,1.862745,2,2.166667,1
271,271,囧囧打人,Aridor,1.0,2021-02-20 16:14:40,296.0,177.0,UNMI (breakthrough),10.0,1.700000,...,2021-07-18 00:00:00,3.0,10,270,26,20,2.032086,3,1.966667,0
274,274,xixo,Point Luna,1.0,2021-03-26 13:41:06,484.0,50.0,Pobolog (breakthrough),6.0,1.666667,...,2021-04-21 00:00:00,2.0,31,184,112,70,2.033333,4,2.045455,1
295,295,钝钝,Saturn Systems,4.0,2020-08-31 15:05:21,60.0,341.0,Point Luna,24.0,2.250000,...,2021-04-25 00:00:00,6.0,3,253,43,3,2.037037,5,2.293103,0


In [ ]:
# final json file
final_df = pd.read_csv("measure_summary_sort.csv")
# entity modification
final_df[["create_no","played_day","total_games","fa_corp_times","most_freq_play_time","highest_score","highest_gens","most_date_times"]] = final_df[["create_no","played_day","total_games","fa_corp_times","most_freq_play_time","highest_score","highest_gens","most_date_times"]].astype(np.int32)
final_df["avg_rank"] = final_df["avg_rank"].round(1)
final_df["avg_gens"] = final_df["avg_gens"].round(1)
final_df[["create_time_text","first_game_text","total_game_text","favorite_corp_text","favorite_time_text","avg_rank_text","avg_gen_text","best_score_text","games_in_one_day_text","achievements"]] = final_df.apply(lambda x:('','','','','','','','','','',),axis=1,result_type='expand')
# for loop
for i,row in final_df.iterrows():
    # 1, user_id, TODO make it to be index, create_no, create_time_text
    if final_df.loc[i,"create_no"] <= 100: final_df.loc[i,"create_time_text"] = "作为登陆火星的先行者，你们为火星的建设奠定了坚实的基础！"
    elif final_df.loc[i,"create_no"] <= 805: final_df.loc[i,"create_time_text"] = "作为火星的第二批居民，你们牵引了无数小行星让火星不再孤寂寒冷。"
    else: final_df.loc[i,"create_time_text"] = "作为火星的新兴势力，感谢你们为火星增添了几分绿意！"

    # 2, first_time, first_game_text
    date_str = final_df.loc[i,"first_time"]
    final_df.loc[i,"first_time"] = date_str[0:4]+'年'+date_str[5:7]+'月'+date_str[8:10]+'日'
    if final_df.loc[i,"first_rank"].astype(np.int32) == 1: final_df.loc[i,"first_game_text"] = "崭露头角，轻取桂冠"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 2: final_df.loc[i,"first_game_text"] = "小试牛刀，施精布略"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 3: final_df.loc[i,"first_game_text"] = "初出茅庐，志在千里"
    elif final_df.loc[i,"first_rank"].astype(np.int32) == 4: final_df.loc[i,"first_game_text"] = "愈战愈勇，砥砺前行"

    # 3, played_day, total_games, total_game_text
    if final_df.loc[i,"total_games"].astype(np.int32) <= 10: final_df.loc[i,"total_game_text"] = "火星好寂寞，下次记得多陪陪人家~"
    elif final_df.loc[i,"total_games"].astype(np.int32) <= 60: final_df.loc[i,"total_game_text"] = "虽然玩得不多，但是改造委员会看到了你的诚意！"
    elif final_df.loc[i,"total_games"].astype(np.int32) <= 120: final_df.loc[i,"total_game_text"] = "改造委员会宣布你是火星年度劳模！"
    else: final_df.loc[i,"total_game_text"] = "地球办公室向你颁发感动火星十大人物证书！"

    # 4, favorite_corp, fa_corp_times
    if final_df.loc[i,"fa_corp_times"].astype(np.int32) >= 10: final_df.loc[i,"favorite_corp_text"] = "日久生情，想必彼此之间已经有羁绊了吧！"
    else: final_df.loc[i,"favorite_corp_text"] = "不要忘记和他一起度过的愉快生活哦~"

    # 5, most_freq_play_time, favorite_time_text
    if final_df.loc[i,"most_freq_play_time"].astype(np.int32) <= 14 and final_df.loc[i,"most_freq_play_time"].astype(np.int32) >= 7: final_df.loc[i,"favorite_time_text"] = "早起的火星人有牌抽！"
    elif final_df.loc[i,"most_freq_play_time"].astype(np.int32) <= 21 and final_df.loc[i,"most_freq_play_time"].astype(np.int32) > 14: final_df.loc[i,"favorite_time_text"] = "真是元气满满的火星人呢"
    else: final_df.loc[i,"favorite_time_text"] = "什么？还有不要睡觉的火星人？你是来自木星的吗？"

    # 6, avg_rank, avg_rank_text
    if final_df.loc[i,"avg_rank"] <= 2: final_df.loc[i,"avg_rank_text"] = "你是出色的技术流！世界政府需要你这样的人才！"
    elif final_df.loc[i,"avg_rank"] <= 3: final_df.loc[i,"avg_rank_text"] = "运气总是起起落落落落落落，我懂你！"
    elif final_df.loc[i,"avg_rank"] <= 4: final_df.loc[i,"avg_rank_text"] = "别灰心！4个人里面只有1个人垫底！"

    # 7, avg_gens, avg_gen_text
    if final_df.loc[i,"avg_gens"] <= 7.2: final_df.loc[i,"avg_gen_text"] = "你是纯纯粹粹的改造党，火卫二不在的第N天，想她"
    elif final_df.loc[i,"avg_gens"] <= 8: final_df.loc[i,"avg_gen_text"] = "你改造拖拖两开花，进可过牌变魔术，退可标动打工人"
    else: final_df.loc[i,"avg_gen_text"] = "你是如假包换的老拖拖，请问AI中心研究的反重力科技成果怎么样了？"

    # 8, highest_corp,highest_score,highest_gens
    if final_df.loc[i,"highest_sort"].astype(np.int32) <= 50: final_df.loc[i,"best_score_text"] = "火星吉尼斯纪录应该留下你的名字！"
    elif final_df.loc[i,"highest_sort"].astype(np.int32) <= 100: final_df.loc[i,"best_score_text"] = "金星总督向你发来祝贺。"
    else: final_df.loc[i,"best_score_text"] = "再接再厉！没事多去研究前哨所转转！"

    # 9 most_date,most_date_times
    date_str = final_df.loc[i,"first_time"]
    final_df.loc[i,"most_date"] = date_str[0:4]+'年'+date_str[5:7]+'月'+date_str[8:10]+'日'
    if final_df.loc[i,"most_date_times"].astype(np.int32) <= 1: final_df.loc[i,"games_in_one_day_text"] = "哦，原来最多也只是开了一局啊，那没事了。"
    elif final_df.loc[i,"most_date_times"].astype(np.int32) <= 3: final_df.loc[i,"games_in_one_day_text"] = "时间管理大师非你莫属！"
    else: final_df.loc[i,"games_in_one_day_text"] = "火星996说的就是你吧！"

    # achievements: play_times_sort,gens_sort,terra_sort,highest_sort,bayes_rank,rank_sort,baye_avg,max_corp
    mapping = ['零','一','二','三','四','五','六','七','八','九','十']
    operated_str = ''
    if final_df.loc[i,"play_times_sort"] <= 10: operated_str = operated_str+("国服第"+mapping[final_df.loc[i,"play_times_sort"]]+"肝帝\n")
    if final_df.loc[i,"gens_sort"] <= 10: operated_str = operated_str+("国服第"+mapping[final_df.loc[i,"gens_sort"]]+"拖拖党\n")
    if final_df.loc[i,"terra_sort"] <= 10: operated_str = operated_str+("国服第"+mapping[final_df.loc[i,"terra_sort"]]+"改造党\n")
    if final_df.loc[i,"rank_sort"] <= 10: operated_str = operated_str+("年度第"+mapping[final_df.loc[i,"rank_sort"]]+"综合实力\n")
    if final_df.loc[i,"max_corp"] == 1: operated_str = operated_str+("国服最强"+final_df.loc[i,"favorite_corp"]+"\n")

    if operated_str == '': 
        final_df.loc[i,"achievements"] = "火星打工人"
    else:
        final_df.loc[i,"achievements"] = operated_str
final_df.to_csv("measure_output.csv")
# final_df.head()

In [ ]:
final_df = final_df.loc[:,["user_id", "create_no", "create_time_text",  "first_time", "first_corp", "first_game_text", "played_day", "total_games", "total_game_text", "favorite_corp", "fa_corp_times", "fa_corp_rank", "most_freq_play_time", "favorite_time_text", "avg_rank", "avg_rank_text", "avg_gens", "avg_gen_text", "highest_corp", "highest_score", "highest_gens", "highest_sort","most_date", "most_date_times", "games_in_one_day_text", "achievements"]]

In [ ]:
json_df = final_df.set_index("user_id", drop=True)
json_df.head()
json_df.to_csv("json_data.csv")
result = json_df.to_json(open("json_data7.json","w",encoding="utf-8"), orient="index", force_ascii=False)